In [1]:
import peachbox

In [2]:
batch_views = peachbox.CassandraDriver()
batch_views.set_keyspace('batch_views')

In [3]:
query = batch_views.execute("SELECT * FROM reviews")

In [4]:
list(query)

[Row(partition_key=u'00a0411d-6c06-b611-e58a-c90242ac1100', product_id=u'B00004RJEJ', review_count=4, review_id=u'-1083712196059441107', score=4, true_as_of_seconds=905472000),
 Row(partition_key=u'7b3715f2-06b7-11e5-8ac9-0242ac11000c', product_id=u'B002AYXT48', review_count=4, review_id=u'2813836952531553091', score=3, true_as_of_seconds=913680000),
 Row(partition_key=u'00e40f3f-1406-b711-e58a-c90242ac1100', product_id=u'B007OWUVDS', review_count=36, review_id=u'-2230019442873608950', score=5, true_as_of_seconds=891648000),
 Row(partition_key=u'28126884-06b9-11e5-8ac9-0242ac11000c', product_id=u'6304798482', review_count=25, review_id=u'2035341071764109935', score=3, true_as_of_seconds=901929600),
 Row(partition_key=u'00cb1793-4206-af11-e58a-c90242ac1100', product_id=u'B0002KPI2S', review_count=1, review_id=u'2420188659791460480', score=5, true_as_of_seconds=911520000),
 Row(partition_key=u'504bde3a-06b8-11e5-8ac9-0242ac11000c', product_id=u'B00004R6RZ', review_count=9, review_id=u'42

In [6]:
query = batch_views.execute("SELECT true_as_of_seconds, review_count FROM reviews WHERE product_id='B0002KPI2S'")
review_counts = map(lambda x: (x.true_as_of_seconds, x.review_count), query)
review_counts                                                                              

[(911520000, 1),
 (911520000, 9),
 (915580800, 1),
 (911520000, 4),
 (911520000, 1),
 (911520000, 4),
 (911520000, 4),
 (915580800, 1),
 (911520000, 9),
 (911520000, 9),
 (911520000, 9),
 (911520000, 1),
 (911520000, 9),
 (915580800, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (911520000, 9),
 (911520000, 1),
 (915580800, 1),
 (911520000, 4),
 (911520000, 9),
 (911520000, 9),
 (915580800, 1),
 (911520000, 9)]

In [7]:
latest_timestamp = max(map(lambda x: x[0], review_counts))
latest_timestamp

915580800

In [11]:
real_time_views = peachbox.CassandraDriver()
real_time_views.set_keyspace('real_time_views')
real_time_views = peachbox.CassandraDriver()
real_time_views.set_keyspace('real_time_views')
real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='B00004U12X'\
                            AND true_as_of_seconds>"+str(latest_timestamp)+" ALLOW FILTERING")

[Row(count=1)]

In [12]:
import kafka.client                                                                                                        
import kafka.producer      
import ujson as json
import time
                                                                                                                           
client   = kafka.client.KafkaClient('localhost:9092')                                                                      
producer = kafka.producer.SimpleProducer(client)                                                                           
                                                                                                                           
message = {'user_id':'A37I5QIB*9UMPD',
           'product_id':'B00004U12X',
           'review':'review text',
           'summary':'Summarizing a great movie',
           'profile_name':'ItsMe',
           'helpfulness':'2/2',
           'time':int(time.time()),
           'score':5.0}
                                                                                                   
producer.send_messages('movie_reviews', json.dumps(message))                                                                      


[ProduceResponse(topic='movie_reviews', partition=0, error=0, offset=12983)]

In [17]:
def review_count(product_id):
    batch_views = peachbox.CassandraDriver()
    batch_views.set_keyspace('batch_views')
    
    real_time_views = peachbox.CassandraDriver()
    real_time_views.set_keyspace('real_time_views')

    query = batch_views.execute("SELECT true_as_of_seconds, review_count\
                                 FROM reviews WHERE product_id='"+product_id+"'")
    review_counts    = map(lambda x: (x.true_as_of_seconds, x.review_count), query)
    latest_timestamp = max(map(lambda x: x[0], review_counts))
    rt_review_counts = real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='"+product_id+"'\
                            AND true_as_of_seconds>"+str(latest_timestamp)+" ALLOW FILTERING")[0].count
    return sum(map(lambda x: x[1], review_counts)) + rt_review_counts

In [18]:
review_count('B00004U12X')

9